## **Imports**

In [14]:
#python -m spacy download de_core_news_sm
import spacy
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, similarities
from gensim.matutils import sparse2full
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
from PyPDF2 import PdfReader

## **Information**

Prefer spacy instead of nltk because it's faster and we want to compute large texts.
Lemmatization and Stemming are redundand and different aproaches. They normaly are not used together. Decided for Lemmatizing because its integrated into spacy module:


Sources:
- Text Similarity Measures in News Articles by Vector Space Model Using NLP (https://link.springer.com/article/10.1007/s40031-020-00501-5)
- Compare documents similarity using Python | NLP (https://dev.to/thedevtimeline/compare-documents-similarity-using-python-nlp-4odp)
- What is gensim.similarities.MatrixSimilarity() function? (https://www.educative.io/answers/what-is-gensimsimilaritiesmatrixsimilarity-function)

In [2]:
documents = [
    "Das ist ein Text.",
    "Ich habe einen Text geschrieben.",
    "Ich habe mehrere Texte geschrieben!",
    "Das sind viele texte. Insgesamt sind es 4."
]

sp = spacy.load("de_core_news_sm")

# **Extract PDF to string**

In [17]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    full_text = ''

    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text = page.extract_text()
        full_text += text

    return full_text

cdu_text = extract_text_from_pdf('cdu_csu.pdf')

with open('cdu_text.txt', 'w', encoding='utf-8') as file:
    file.write(cdu_text)

print(cdu_text)

spd_text = extract_text_from_pdf('spd.pdf')

with open('spd_text.txt', 'w', encoding='utf-8') as file:
    file.write(cdu_text)
    
print(spd_text)

party_docs = []

party_docs.append(cdu_text)
party_docs.append(spd_text)

   Das Programm für Stabilität   
   und Erneuerung.  
  GEMEINSAM FÜR EIN MODERNES DEUTSCHLAND.   Seite 1 von 139 Inhaltsverzeichnis 
 
Einleitung ...................................................................................................................  4 
 
1. Neue Verantwortung Deutschlands in der Welt – aus Überzeugung für Frieden, 
Freiheit und Menschenrechte ...............................................................................  6 
1.1. Stärkung der werte- und regelbasierten internationalen Ordnung .........................  6 
1.2. Deutschland als Stabilitätsanker in der globalen Welt ...............................................  7 
1.3. Aufbruch für die transatlantische Partnerschaft .........................................................  8 
1.4. Stabilität in ganz Europa ..................................................................................................  9 
1.5. Besondere Verantwortung Deutschlands gegenüber Israel ......................

# **Preprocessing**

## **1. Tokenization**

In [3]:
tokenized_docs = []
for doc in documents:
    doc_tokens = sp(doc)
    tokenized_tokens = [token.text for token in doc_tokens]
    tokenized_docs.append(tokenized_tokens)

print("Tokenized Documents:")
for doc_tokens in tokenized_docs:
    print(doc_tokens)

Tokenized Documents:
['Das', 'ist', 'ein', 'Text', '.']
['Ich', 'habe', 'einen', 'Text', 'geschrieben', '.']
['Ich', 'habe', 'mehrere', 'Texte', 'geschrieben', '!']
['Das', 'sind', 'viele', 'texte', '.', 'Insgesamt', 'sind', 'es', '4.']


## **2. Lammatize**

In [4]:
lemmatized_docs = []
for doc_tokens in tokenized_docs:
    lemmatized_tokens = [token.lemma_ for token in sp(' '.join(doc_tokens))]
    lemmatized_docs.append(lemmatized_tokens)

print("\nLemmatized Documents:")
for lemmatized_tokens in lemmatized_docs:
    print(lemmatized_tokens)


Lemmatized Documents:
['der', 'sein', 'ein', 'Text', '--']
['ich', 'haben', 'ein', 'Text', 'schreiben', '--']
['ich', 'haben', 'mehrere', 'Text', 'schreiben', '--']
['der', 'sein', 'vieler', 'Text', '--', 'insgesamt', 'sein', 'es', '4.']


## **3. Remove Stop Words**

In [5]:
lemmatized_docs_no_stopwords = []

for lemmatized_tokens in lemmatized_docs:
    lemmatized_tokens_no_stopwords = [token for token in lemmatized_tokens if not sp.vocab[token].is_stop]
    lemmatized_docs_no_stopwords.append(lemmatized_tokens_no_stopwords)

print("\nLemmatized Documents without Stop Words:")
for lemmatized_tokens in lemmatized_docs_no_stopwords:
    print(lemmatized_tokens)


Lemmatized Documents without Stop Words:
['Text', '--']
['Text', 'schreiben', '--']
['mehrere', 'Text', 'schreiben', '--']
['vieler', 'Text', '--', 'insgesamt', '4.']


## **4.Eleminate Puctation Marks & Numbers**

In [6]:
lemmatized_docs_no_stopwords_punct_nums = []

for lemmatized_tokens in lemmatized_docs:
    lemmatized_tokens_no_stopwords = [token for token in lemmatized_tokens if not sp.vocab[token].is_stop]
    
    # Remove punctuation tokens
    lemmatized_tokens_no_punct = [token for token in lemmatized_tokens_no_stopwords if not sp.vocab[token].is_punct]
    
    # Remove number tokens
    lemmatized_tokens_no_nums = [token for token in lemmatized_tokens_no_punct if not sp.vocab[token].like_num]
    
    lemmatized_docs_no_stopwords_punct_nums.append(lemmatized_tokens_no_nums)

print("\nLemmatized Documents without Stop Words, Punctuation, and Numbers:")
for tokens in lemmatized_docs_no_stopwords_punct_nums:
    print(tokens)



Lemmatized Documents without Stop Words, Punctuation, and Numbers:
['Text']
['Text', 'schreiben']
['mehrere', 'Text', 'schreiben']
['vieler', 'Text', 'insgesamt']


## **Preprocessing Class**

In [18]:
import spacy

class TextPreprocessor:
    def __init__(self):
        self.nlp = spacy.load("de_core_news_sm")
    
    def tokenize_documents(self, documents):
        tokenized_docs = []
        for doc in documents:
            doc_tokens = self.nlp(doc)
            tokenized_tokens = [token.text for token in doc_tokens]
            tokenized_docs.append(tokenized_tokens)
        return tokenized_docs
    
    def lemmatize_documents(self, tokenized_docs):
        lemmatized_docs = []
        for doc_tokens in tokenized_docs:
            lemmatized_tokens = [token.lemma_ for token in self.nlp(' '.join(doc_tokens))]
            lemmatized_docs.append(lemmatized_tokens)
        return lemmatized_docs
    
    def remove_stopwords_punctuations_numbers(self, lemmatized_docs):
        clean_docs = []
        for lemmatized_tokens in lemmatized_docs:
            lemmatized_tokens_no_stopwords = [token for token in lemmatized_tokens if not self.nlp.vocab[token].is_stop]
            lemmatized_tokens_no_punct = [token for token in lemmatized_tokens_no_stopwords if not self.nlp.vocab[token].is_punct]
            lemmatized_tokens_no_nums = [token for token in lemmatized_tokens_no_punct if not self.nlp.vocab[token].like_num]
            clean_docs.append(lemmatized_tokens_no_nums)
        return clean_docs

documents = [
    """Unser Unions-Versprechen: Wir arbeiten für ein modernes Europa, das weltpolitikfähig ist, um
die globalen Herausforderungen gemeinsam zu meistern. Dafür muss Europa handlungsfähiger,
mutiger und entschlossener werden. Denn nur wenn es Europa gut geht, geht es auch Deutschland gut.
Europa wird herausgefordert – von innen und von außen. Innerhalb Europas setzen Populisten
von links und rechts die europäische Demokratie unter Druck. Zusätzlich erschweren Nationalismus und Eigeninteressen einiger EU-Mitgliedsstaaten immer wieder gemeinsame europäische
Lösungen oder verhindern ein Auftreten der EU mit einer Stimme. Und schließlich ist die EU in
zentralen Bereichen, wie etwa der Verteidigungspolitik, nicht so handlungsfähig, wie wir uns das
wünschen.
Auch von außen sehen wir den europäisch-abendländischen Leitgedanken der Demokratie wie
auch der Sozialen Marktwirtschaft unter Druck und im Wettbewerb mit konkurrierenden Gesellschafts- und Wirtschaftsmodellen. Freier Welthandel mit offenen Märkten, der uns Wohlstand gebracht hat, ist keine Selbstverständlichkeit mehr.
Unsere Antwort auf diese Herausforderung lautet: Mehr Europa! Denn nur gemeinsam mit unseren europäischen Partnern werden wir die Herausforderungen meistern. Dafür brauchen wir
schnellere und dynamischere Entscheidungen dort, wo es europäische Lösungen und entschlossenes Handeln auf internationaler Ebene braucht. Dabei gilt die Formel: Nicht jedes Problem in
Europa ist ein Problem für Europa. Gleichzeitig werden wir das Modernisierungsjahrzehnt auch
auf Europa erstrecken: Wir investieren in Technologien und Innovationen, damit Europas Wirtschaft auch in Zukunft Garant für Wohlstand, Arbeitsplätze und Nachhaltigkeit bleibt. Wir investieren in Europas Sicherheit, ob nach innen oder außen, damit auch unsere Kinder und Enkel
in Europa in Frieden, Freiheit und Sicherheit leben können""",

    """Die AfD steht für die Freiheit und Selbstbestimmung der
europäischen Nationen. Wir bekennen uns zu einem
Europa der Vaterländer als einer Gemeinschaft
souveräner Staaten, die auf all jenen Gebieten
zusammenarbeiten, die gemeinsam besser gestaltet
werden können. Dazu gehört insbesondere ein freier
Handel mit fairem Wettbewerb.
Eine staatsähnliche Europäische Union, wie sie von den
etablierten Parteien angestrebt wird, halten wir im Sinne
eines prosperierenden und friedlichen Europas für
kontraproduktiv. Selbstverantwortliche und von
lebendigen Demokratien gestaltete Nationalstaaten sind
durch übernationale Einrichtungen nicht ersetzbar.
Eine Gruppe benachbarter Staaten kann sehr gut auf
völkerrechtlicher Basis konstruktiv und friedlich
kooperieren. Der Versuch jedoch, aus derzeit 27 oder
noch mehr Staaten mit jeweils eigenen Sprachen,
Kulturen und historischen Erfahrungen einen wie auch
immer ausgestalteten Gesamtstaat zu bilden, muss
scheitern. Ein solches Gebilde verfügt weder über ein
Staatsvolk, noch über das erforderliche Mindestmaß an
kultureller Identität, welche notwendige Voraussetzungen für gelingende Staaten sind. Wir wollen den
souveränen, demokratischen Nationalstaat erhalten.
Nur dort kann Volkssouveränität gelebt werden, die
Mutter und das Herzstück der Demokratie.Eine Union europäischer Staaten wird nur dann eine
Zukunft haben, wenn es gelingt, dem sich immer
schneller drehenden Rad der Entdemokratisierung und
Zentralisierung in die Speichen zu greifen, bevor die
heutige EU durch die Pervertierung ihrer Gründungsidee an sich selbst zugrunde geht. Die wirtschaftsgeschichtlich ungewöhnliche Idee einer Einheitswährung für wirtschaftlich völlig unterschiedlich
entwickelte Staaten ist gescheitert. Es handelte sich um
eine politische Wunschvorstellung, die mit
ökonomischen Gesetzen nicht in Einklang zu bringen ist.
Mit der Einführung des sogenannten „CoronaWiederaufbaupaktes“ wurde die Transferunion in eine
neue Dimension gehoben. Diese Transferunion steht
nicht nur im Widerspruch zu den europäischen
Verträgen und den Versprechen der deutschen Politiker,
sondern wird den Abstieg aller europäischen
Volkswirtschaften und Konflikte zwischen den Staaten
zur Folge haben.
Ein vergleichbares Versagen zeigt die EU in der jahrelang
anhaltenden Migrationskrise. Auch das Krisenmanagement in der Corona-Krise war verheerend.
Die Fehlentwicklungen in der Klima- und Energiepolitik
treibt die EU durch unvorstellbar kostspielige Gesetzesund Subventionspakete voran. Das langfristige Unheil
dieser Verschuldungs- und Umverteilungs-Eskapaden
wird vor allem den deutschen Steuerzahler treffen. Die Vehemenz, mit welcher die Europäische Union die
Transformation zum planwirtschaftlichen Superstaat
in den letzten Jahren vorangetrieben hat, hat uns zu der
Erkenntnis gebracht, dass sich unsere grundlegenden
Reformansätze in dieser EU nicht verwirklichen lassen.
Wir halten einen Austritt Deutschlands aus der
Europäischen Union und die Gründung einer neuen
europäischen Wirtschafts- und Interessengemeinschaft
für notwendig""",

"""Grenzenlos reisen, arbeiten, studieren oder leben. Die Europäische Union (EU) hat das Leben von
Millionen von Menschen geprägt, neue Möglichkeiten und Freiheiten eröffnet und den unermesslichen Wert kultureller Vielfalt für unsere Gesellschaften für viele erlebbar gemacht. Sie hat gezeigt, dass wir gemeinsam mehr erreichen können. Darauf wird es in Zukunft ganz besonders
ankommen. 
In einem von globalem Wettbewerb geprägten Umfeld können wir unsere europäischen Werte
und Interessen nur behaupten, wenn Europa nach innen geeint und nach außen handlungsfähig
ist. Wir wollen die Freiheit und Rechtstaatlichkeit in Europa schützen und die EU zur modernsten
Demokratie der Welt machen. Wir wollen, dass Europa auch beim Klimaschutz Vorreiter wird. Wir
wollen ein digital souveränes Europa auf der Basis einer wertebasierten digitalen Wirtschaft. Mit
Investitionen in unsere gemeinsame Wirtschafts- und Innovationskraft stärken wir Europa als den
modernsten, sozialsten, nachhaltigsten und wettbewerbsfähigsten Wirtschaftsraum der Welt und
sichern so die Grundlagen unseres Wohlstands. Damit schaffen wir die Voraussetzungen für ein
souveränes Europa, das für soziale Gerechtigkeit, Wohlstand und Menschenrechte steht und sich
geschlossen für eine gerechtere, friedlichere und nachhaltigere Welt einsetzt. 
Nur miteinander werden wir eine humanitäre und solidarische Flüchtlingspolitik gewährleisten.
Demokratie, Freiheit und Rechtsstaatlichkeit sind das Fundament unserer Gemeinschaft. Wir werden es nicht zulassen, dass nationalistischer Hass und populistische Hetze Europa spalten. 
Wir meinen es ernst mit der europäischen Solidarität. Wir haben zu Beginn der Pandemie schnell
und kraftvoll gehandelt und den größten Wiederaufbaufonds der Geschichte der Europäischen
Union auf den Weg gebracht – eine Solidarleistung, die die sozialen Folgen der Corona-Krise abmildert und die gleichzeitig den sozial-ökologischen Wandel vorantreibt und Innovationen fördert.
Europa bekommt dafür weltweite Anerkennung. Auf dieser Basis wollen wir neues Vertrauen in
Europa aufbauen und eine wirtschaftliche und politische Spaltung der EU verhindern. """,
"""Europa muss bereit sein, die großen Herausforderungen
unserer Zeit zu bewältigen – die Folgen der Coronapandemie, den Klimawandel, Terrorismus und Migration. Wir Freie
Demokraten wollen eine außenpolitisch starke EU, die ihre
Werte, ihre Interessen und Souveränität schützt sowie sich
autokratischem Machtstreben entgegenstellt. Mit mutigen Reformen ihrer Aufgaben, Arbeitsweise und Institutionen wollen
wir die EU nach innen demokratisch und wirtschaftlich stark
sowie nach außen handlungsfähig machen. So wird die EU
zu einem echten Global Player.
Zukunftskonferenz für neuen Schwung in Europa
nutzen
Wir Freie Demokraten unterstützen die Konferenz zur Zukunft
Europas. In diesem politischen Gremium sollen in einem einjährigen Prozess alle europäischen Institutionen zusammenkommen, um die Prioritäten für die EU mit Bürgerinnen und
Bürgern aus allen Mitgliedstaaten zu diskutieren. Wir wollen,
dass sich die Konferenz auf zentrale Politikfelder konzentriert,
die für unsere gemeinsame Zukunft langfristig von Relevanz
sind. Dafür müssen Leitlinien, Ziele und Prioritäten definiert
werden. Für uns geht es darum, die EU bei der Bewältigung
von Pandemien robuster aufzustellen, Europa als Chancenkontinent zu entwickeln und als Fortschrittsmotor zu mobilisieren. Außerdem wollen wir die EU institutionell reformieren,
um sie bürgernäher und handlungsfähiger zu machen. Das
schließt auch Vertragsänderungen ein. Diese müssen anschließend von den Mitgliedstaaten und von den EU-Institutionen
angemessen umgesetzt werden.
Für eine gemeinsame Verfassung der Europäischen
Union als Bundesstaat
Wir Freie Demokraten wollen nach Abschluss der Konferenz
zur Zukunft Europas einen Verfassungskonvent einberufen.
Dieser Konvent sollte einer dezentral und föderal verfassten
Union eine rechtsverbindliche Verfassung mit einem Grundrechtekatalog und starken Institutionen geben. Über die neue
Europäische Verfassung sollen die Bürgerinnen und Bürger
der EU in einer gemeinsamen europäischen Volksabstimmung
entscheiden und damit die Grundlage für einen föderal und
dezentral verfassten Europäischen Bundesstaat schaffen.
Dieser Weg ist das erklärte Gegenmodell zum Rückfall
Europas in nationalstaatliche Kleinstaaterei einerseits oder
die Schaffung eines zentralisierten europäischen Superstaats
andererseits. Bis dahin möchten wir, dass die europäischeIntegration parallel durch ein „Europa der verschiedenen Geschwindigkeiten“ vertieft wird.
Für ein starkes EU-Parlament und mehr
Transparenz
Wir Freie Demokraten fordern institutionelle Reformen für
mehr Transparenz und Effizienz in der EU. Das Europäische Parlament soll nach einem einheitlichen Wahlrecht mit
staatenübergreifenden Listen und Spitzenkandidatinnen und
Spitzenkandidaten gewählt werden. Es muss zu einem Vollparlament mit Initiativrecht aufgewertet werden. Das Europäische Parlament soll einen festen Tagungsort haben und selbst
über seinen Sitz entscheiden können. Kommissionspräsidentin
oder -präsident wird die Spitzenkandidatin oder der Spitzenkandidat, die oder der im EU-Parlament eine Mehrheit der
Stimmen auf sich vereint. Das Parlament kann ihr oder ihm
durch die Mehrheit seiner Mitglieder das Misstrauen aussprechen und eine andere Person zum Kommissionspräsidenten
wählen. Das Vorschlagsrecht für die übrigen Kommissarinnen
und Kommissare liegt bei der Kommissionspräsidentin beziehungsweise beim -präsidenten und das Parlament muss die
Vorschläge einzeln bestätigen. Die EU-Kommission sollte auf
höchstens 18 Kommissarinnen und Kommissare verkleinert
werden. Hierbei müssen klare und einfach zurechenbare
Ressorts vergeben werden, die den EU-Zuständigkeiten entsprechen. Der Rat der Europäischen Union und seine Untergruppierungen sollen öffentlich tagen.
Offene Strategische Souveränität der EU anstreben
Wir Freie Demokraten unterstützen das Ziel der Europäischen
Union, eine größere strategische Souveränität zu erreichen.
Für uns bedeutet dies in erster Linie eigenständige Handlungsfähigkeit. Mit den erforderlichen Mitteln ausgestattet, könnte
die EU in Zukunft ihre eigenen Interessen und Werte besser
durchsetzen und in wichtigen Bereichen wie zum Beispiel
Energieversorgung, Rohstoffimporte und digitale Technologie
weniger abhängig und verwundbar werden. In der Handelsund Entwicklungspolitik muss die EU die eigenen Stärken
strategischer einsetzen und in der Gemeinsamen Sicherheitsund Verteidigungspolitik (GSVP) eigene militärische Fähigkeiten entwickeln. Das steht nicht im Widerspruch zur transatlantischen Partnerschaft und zur NATO, sondern erhöht das
Gewicht der EU, als Partnerin auf Augenhöhe einen Beitrag
zur liberalen Weltordnung zu leisten. Der Wunsch nach
strategischer Souveränität darf aber weder zu Protektionismus
noch zu einer Selbstisolation führen.
Echte Gemeinsame Außen- und Sicherheitspolitik in
Europa
Wir Freie Demokraten wollen eine Gemeinsame Außen- und
Sicherheitspolitik (GASP) der EU, die den Namen auch
verdient. Die Europäische Union muss international schneller handlungsfähig sein und nach außen mit einer Stimme
sprechen. Wir fordern deshalb, dass die Einstimmigkeit im"""
]


preprocessor = TextPreprocessor()

# Tokenization
tokenized = preprocessor.tokenize_documents(party_docs)

# Lemmatization
lemmatized = preprocessor.lemmatize_documents(tokenized)

# Removing Stopwords, Punctuation, and Numbers
cleaned = preprocessor.remove_stopwords_punctuations_numbers(lemmatized)

# Print cleaned documents
print("\nCleaned Documents:")
for tokens in cleaned:
    print(tokens)



Cleaned Documents:
['    ', 'Programm', 'Stabilität', '  \n    ', 'Erneuerung', ' \n   ', 'GEMEINSAM', 'MODERNES', 'DEUTSCHLAND', '   ', 'Seite', 'Inhaltsverzeichnis', '\n \n ', 'Einleitung', '  ', '\n \n ', 'neu', 'Verantwortung', 'Deutschland', 'Welt', 'Überzeugung', 'Frieden', '\n ', 'Freiheit', 'Menschenrecht', '  ', '\n ', 'Stärkung', 'werte-', 'regelbasiert', 'international', 'Ordnung', '  ', '\n ', 'Deutschland', 'Stabilitätsanker', 'global', 'Welt', '  ', '\n ', 'Aufbruch', 'transatlantisch', 'Partnerschaft', '  ', '\n ', 'Stabilität', 'Europa', '  ', '\n ', 'besonderer', 'Verantwortung', 'Deutschland', 'Israel', '  ', '\n ', 'neu', 'Aufmerksamkeit', 'asiatisch-pazifisch', 'Raum', '  ', '\n ', 'moderne', 'voll', 'einsatzbereit', 'Bundeswehr', '  ', '\n ', 'nachhaltig', 'Entwicklung', 'Welt', '  ', '\n ', 'Entwicklungspartnerschaft', 'Afrika', '  ', '\n ', '  ', 'international', 'Klimaschutz', 'Bewahrung', 'Schöpfung', '\n \n ', 'neu', 'Weltpolitikfähigkeit', 'Leidenschaft', 's

# **Representation Scheme**

## **Bag of Words**

In [19]:
def create_bow_representation(preprocessed_docs):
    dictionary = corpora.Dictionary(preprocessed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]
    return dictionary, bow_corpus

bow_dictionary, bow_corpus = create_bow_representation(cleaned)

# Display bow representation
print("\nBag-of-Words Representation:")
num_terms = len(bow_dictionary)
bow_matrix = [sparse2full(doc, num_terms) for doc in bow_corpus]
bow_df = pd.DataFrame(bow_matrix, columns=[bow_dictionary[i] for i in range(num_terms)])

display(bow_df)

print(bow_dictionary)
print(bow_corpus)


Bag-of-Words Representation:


,,\n,\n \n,\n,\n,,\n,\n,\n,,...,überleben,übermäßig,überprüfbar,übersetzten,überteueren,überteuert,übertreffen,überwiegend,überwind,überwindba-
0,1.0,988.0,10.0,40.0,5.0,1.0,3674.0,206.0,17.0,1350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2285.0,1.0,3.0,0.0,0.0,32.0,0.0,0.0,56.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Dictionary<12124 unique tokens: ['', '\n ', '\n \n ', '\n  ', '\n   ']...>
[[(0, 1), (1, 988), (2, 10), (3, 40), (4, 5), (5, 1), (6, 3674), (7, 206), (8, 17), (9, 1350), (10, 1), (11, 27), (12, 4), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 4), (51, 1), (52, 1), (53, 1), (54, 4), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 6), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 6), (79, 3), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 3), (86, 1), (87, 7), (88, 1), (89, 3), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 11), (101, 1)

## **TF-IDF**

In [20]:
def create_tfidf_representation(preprocessed_docs):
    # Joining tokenized documents to form a list of strings
    tokenized_texts = [' '.join(doc) for doc in preprocessed_docs]
    
    # Creating TF-IDF vectorizer and fitting on the tokenized documents
    ifidf_vectorizer = TfidfVectorizer()
    tfidf_representation = ifidf_vectorizer.fit_transform(tokenized_texts)
    
    feature_names = ifidf_vectorizer.get_feature_names_out()
    
    return tfidf_representation, feature_names

tfidf_corpus, tfidf_dictionary = create_tfidf_representation(cleaned)

tfidf_df = pd.DataFrame(tfidf_corpus.toarray(), columns=tfidf_dictionary)

# Display the TF-IDF DataFrame
print("\nTF-IDF Representation:")
display(tfidf_df)


TF-IDF Representation:


,120,15,18,1seite,2020er,2021basierend,2021bei,2021das,2021dass,2021daten,...,übung,übungsleiter,ück,üfen,ühren,ür,ürg,ürger,ürgerinn,ützt
0,0.001866,0.000000,0.001866,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003732,0.001866,0.001866,0.003732,0.005597,0.018658,0.003732,0.001866,0.001866,0.001866
1,0.000000,0.002988,0.000000,0.01195,0.002988,0.002988,0.002988,0.002988,0.002988,0.002988,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## **Doc2Vec**

In [21]:
def create_doc2vec_representation(preprocessed_docs):
    tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(preprocessed_docs)]
    
    # hyperparameters should be optimized
    max_epochs = 100
    vec_size = 100
    alpha = 0.025
    
    # Train Doc2Vec model
    model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=1)
    model.build_vocab(tagged_data)
    
    for epoch in range(max_epochs):
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= 0.0002
        model.min_alpha = model.alpha
    
    doc2vec_representation = [model.infer_vector(doc.words) for doc in tagged_data]
    
    return doc2vec_representation

doc2vec_corpus = create_doc2vec_representation(cleaned)

# Convert Doc2Vec corpus to a DataFrame
doc2vec_df = pd.DataFrame(doc2vec_corpus)

# Display the Doc2Vec DataFrame
print("\nDoc2Vec Representation:")
display(doc2vec_df)


Doc2Vec Representation:


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-3.684729,0.868359,-4.607926,2.240309,0.961766,-4.341405,-2.663262,-1.748517,-2.215498,0.134736,...,-3.928220,5.420404,-1.349453,-4.792066,-1.377483,4.559233,2.885500,1.956194,0.768170,2.194892
1,-3.922319,-0.127321,-5.316321,1.194968,0.011690,-4.714215,-2.576444,0.019915,-1.641209,0.605952,...,-4.703799,5.044076,0.783849,-5.289896,-1.254255,4.116807,4.727106,1.398352,-0.028299,2.646427


# **Similarity Measures**

## **Cosine**

In [27]:
def calculate_cosine_similarity(corpus, similarity_type='bow'):
    if similarity_type == 'bow':
        index = similarities.MatrixSimilarity(corpus) #calculates automaticly cosine similarity
    elif similarity_type == 'tfidf' or similarity_type == 'doc2vec':
        index = cosine_similarity(corpus)
    else:
        raise ValueError("Invalid similarity_type. Choose 'bow', 'doc2vec' or 'tfidf'.")
    
    return index

bow_similarity_index = calculate_cosine_similarity(bow_corpus, similarity_type='bow')
print("\nCosine Similarity (BoW Representation):")
print(bow_similarity_index)
for i, sims in enumerate(bow_similarity_index):
    print(f"Document {i + 1}: {sims}")


tfidf_similarity_index = calculate_cosine_similarity(tfidf_corpus, similarity_type='tfidf')
print("\nCosine Similarity (TF-IDF Representation):")
for i, sims in enumerate(tfidf_similarity_index):
    print(f"Document {i + 1}: {sims}")


doc2vec_similarity_index = calculate_cosine_similarity(doc2vec_corpus, similarity_type='doc2vec')
print("\nCosine Similarity (Doc2Vec Representation):")
for i, sims in enumerate(doc2vec_similarity_index):
    print(f"Document {i + 1}: {sims}")


Cosine Similarity (BoW Representation):
MatrixSimilarity<2 docs, 12124 features>
Document 1: [1.0000004 0.2798081]
Document 2: [0.2798081 0.9999996]

Cosine Similarity (TF-IDF Representation):
Document 1: [1.         0.68009851]
Document 2: [0.68009851 1.        ]

Cosine Similarity (Doc2Vec Representation):
Document 1: [0.9999998 0.9579389]
Document 2: [0.9579389  0.99999976]


## **Eucliadian for bow**

In [30]:
def calculate_euclidean_similarity(corpus):
    num_terms = max(token_id for doc in corpus for token_id, _ in doc) + 1
    matrix = [sparse2full(doc, num_terms) for doc in corpus]
    
    num_docs = len(matrix)
    similarity_matrix = np.zeros((num_docs, num_docs))
    for i, doc1 in enumerate(matrix):
        for j, doc2 in enumerate(matrix):
            distance = np.linalg.norm(doc1 - doc2) 
            similarity_matrix[i, j] = 1 / (1 + distance)
    
    return similarity_matrix

euclidean_similarity_bow = calculate_euclidean_similarity(bow_corpus)
print("Euclidean Similarity Matrix (bow representation):")
print(euclidean_similarity_bow)

Euclidean Similarity Matrix (bow representation):
[[1.0000000e+00 2.4279398e-04]
 [2.4279398e-04 1.0000000e+00]]


In [29]:
from sklearn.metrics.pairwise import euclidean_distances

def calculate_distance(corpus, distance_type='bow'):
    if distance_type == 'tfidf' or distance_type == 'doc2vec':
        distance = euclidean_distances(corpus)
    else:
        raise ValueError("Invalid distance_type. Choose 'bow', 'doc2vec', or 'tfidf'.")
    
    return distance


# Calculate Euclidean distances for TF-IDF representation
tfidf_distance_index = calculate_distance(tfidf_corpus, distance_type='tfidf')
print("\nEuclidean Distances (TF-IDF Representation):")
for i, dists in enumerate(tfidf_distance_index):
    print(f"Document {i + 1}: {dists}")

# Assuming 'doc2vec_corpus' contains your Doc2Vec representations
# Calculate Euclidean distances for Doc2Vec representation
doc2vec_distance_index = calculate_distance(doc2vec_corpus, distance_type='doc2vec')
print("\nEuclidean Distances (Doc2Vec Representation):")
for i, dists in enumerate(doc2vec_distance_index):
    print(f"Document {i + 1}: {dists}")



Euclidean Distances (TF-IDF Representation):
Document 1: [0.         0.79987685]
Document 2: [0.79987685 0.        ]

Euclidean Distances (Doc2Vec Representation):
Document 1: [ 0.       10.661894]
Document 2: [10.661894  0.      ]
